In [ ]:
# all data sourced from https://www.kaggle.com/hm-land-registry/uk-housing-prices-paid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import linear_model
""" important note conveyancy takes 6-8 weeks - this means that from accepting offer to 
Date of Transfer could be two months different - this will be accounted for later in document"""

In [ ]:
# Load data
housing_df = pd.read_csv('price_paid_records.csv')

In [ ]:
# Display head of data
housing_df.head()

In [ ]:
#D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other 
def property_filter_out(data,col,filter_out=''):
    for item in filter_out:
        data = data[data[col]!=item]
    return data

In [ ]:
# Convert the date column to actual datetime + create month/year columns
housing_df['Date of Transfer'] = pd.to_datetime(housing_df['Date of Transfer'],dayfirst=True)
housing_df['Year'] = housing_df['Date of Transfer'].dt.year
housing_df['Month'] = housing_df['Date of Transfer'].dt.month
# filter out data we don't want
housing_df = property_filter_out(housing_df,'Property Type','TFO')

In [ ]:
# Create a pivot table of data and select only the last 10 years
housing_pivot = pd.pivot_table(housing_df,values='Price',index='Month',columns='Year',aggfunc='mean')
time_range = np.arange(2006,2017)
housing_pivot = housing_pivot[time_range]

In [ ]:
# Plot data
plt.figure(figsize=(10,5))

plt.subplot(1,1,1)
plt.plot(housing_pivot)

plt.legend(time_range,loc=4)
plt.xlabel('Month Jan-Dec')
plt.ylabel("Price - pounds")
plt.title("Price vs Month")
plt.show()

In [ ]:
# Take the mean across years for each month - does price really vary by month?
avg_across_years = np.empty((12,1))

for i in range(0,12):
    avg_across_years[i,0] = np.mean(np.array(housing_pivot)[i,:])
plt.plot(avg_across_years)
plt.ylabel('Price - Pounds')
plt.xlabel('Month Jan - Dec')
plt.show()

In [ ]:
# is this more interesting by region?